# "Настройка" BEIR

In [3]:
!pip install datasets
!pip install beir
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 46.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.9 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63550 sha256=024cd4916fb599b1497620aebc0571a12bd60672bac0d3d6e0cf992126428c1c
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a6183af09d389459195ff5f854862
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=293160 sha256=0b376874a16b353243c5f93165fa2fa7d721198d9a4f2a27fcaa7ba598dcde58
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built beir pytrec_eval


In [ ]:
#!git clone git@github.com:kngrg/rus_beir.git

In [ ]:
import os
import gdown
import zipfile


url = "https://drive.google.com/uc?export=download&id=1RkR7YQ2AewNg96MmHF7sksCC_oxxob5V"
out_dir = os.path.join(os.getcwd(), "datasets")
os.makedirs(out_dir, exist_ok=True)
output_file_path = os.path.join(out_dir, "rus-mmarco.zip")
gdown.download(url, output_file_path, quiet=False)

zip_path = os.path.join(out_dir, "rus-mmarco.zip")
print(zip_path)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(out_dir)
print(f"Dataset extracted here: {out_dir}")

In [ ]:
# Загрузка через HF
from beir.datasets.data_loader_hf import HFDataLoader
corpus, queries, qrels = HFDataLoader(hf_repo="kngrg/rus-mmarco", hf_repo_qrels="kngrg/rus-mmarco/qrels", streaming=False,
                                                  keep_in_memory=False).load(split='train')


In [ ]:
from beir.datasets.data_loader import GenericDataLoader

data_path = "datasets/rus-mmarco"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="dev") # or split = "train"

# ElasticSearch

In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time

# Sleep for few seconds to let the instance start.
time.sleep(20)

In [ ]:
%%bash

ps -ef | grep elasticsearch

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

In [ ]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

#### Provide parameters for elastic-search
hostname = "localhost"
index_name = "mmarco"
initialize = True # True, will delete existing index with same name and reindex all documents

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize, language="russian")
retriever = EvaluateRetrieval(model)

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)

In [ ]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

In [ ]:
metrics = {"ndcg": ndcg, "_map": _map, "recall": recall, "precision": precision}
for metric in metrics.keys():
    for it_num, it_val in zip(metrics[metric], metrics[metric].values()):
        print(it_num, it_val )
    print('\n')
